# PART 1 - PRELIMINARY AND EXPLORATORY ANALYSIS OF STATION_CSV AND STATION_DAY CSV

# Purpose
<b>Finding meaningfull insights about AQI from diffrent stations in india. 

https://www.diva-portal.org/smash/get/diva2:1681590/FULLTEXT02 in this reasearch paper the creators have only dumped the data into their machine learning models but did not explore the data itself. </b>

# what is AQI?

<b> The Air Quality Index (AQI) is used for reporting daily air quality. It tells you how clean or polluted your air is, and what associated health effects might be a concern for you. The AQI focuses on health effects you may experience within a few hours or days after breathing polluted air. </b>

# 1. Imports

In [ ]:
import pandas as pd
import mysql.connector
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import geopandas as gpd
import folium
%matplotlib inline

# ________________________________________________________________________________
# 2. showing columns and heads of station and station_day

In [ ]:
# Establish connection to MySQL database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="3263",
    database="air_quality_db",
    allow_local_infile=True
)
cursor = conn.cursor()

# Creating database
cursor.execute("CREATE DATABASE IF NOT EXISTS air_quality_db;")
cursor.execute("USE air_quality_db;")

# Drop tables if they exist
cursor.execute("DROP TABLE IF EXISTS station_day;")
cursor.execute("DROP TABLE IF EXISTS stations;")


In [ ]:
cursor.execute("""
    CREATE TABLE stations (
        StationId VARCHAR(20) PRIMARY KEY,
        StationName VARCHAR(255),
        City VARCHAR(100),
        State VARCHAR(100),
        Status VARCHAR(20)
    );
""")

# Create the 'station_day' table
cursor.execute("""
    CREATE TABLE station_day (
        StationId VARCHAR(10),
        Date DATE,
        PM2_5 FLOAT NULL,
        PM10 FLOAT NULL,
        NO FLOAT NULL,
        NO2 FLOAT NULL,
        NOx FLOAT NULL,
        NH3 FLOAT NULL,
        CO FLOAT NULL,
        SO2 FLOAT NULL,
        O3 FLOAT NULL,
        Benzene FLOAT NULL,
        Toluene FLOAT NULL,
        Xylene FLOAT NULL,
        AQI FLOAT NULL,
        AQI_Bucket VARCHAR(50),
        FOREIGN KEY (StationId) REFERENCES stations(StationId)
    );
""")

print("Tables created successfully.")

In [ ]:
# Enable LOCAL INFILE for MySQL
cursor.execute("SET GLOBAL local_infile = 1;")

# Load data into 'stations' table (Handle NULL values)
cursor.execute("""
    LOAD DATA LOCAL INFILE '/Users/pulkitsoni/Air_Quality_Analysis_and_Prediction_for_Indian_Cities_and_States/stations.csv'
    INTO TABLE stations
    FIELDS TERMINATED BY ',' 
    ENCLOSED BY '"' 
    LINES TERMINATED BY '\n'
    IGNORE 1 ROWS
    (StationId, StationName, City, State, Status)
    SET StationId = NULLIF(StationId, ''),
        StationName = NULLIF(StationName, ''),
        City = NULLIF(City, ''),
        State = NULLIF(State, ''),
        Status = NULLIF(Status, '');
""")

# Load data into 'station_day' table (Handle NULL values)
cursor.execute("""
    LOAD DATA LOCAL INFILE '/Users/pulkitsoni/Air_Quality_Analysis_and_Prediction_for_Indian_Cities_and_States/station_day.csv'
    INTO TABLE station_day
    FIELDS TERMINATED BY ',' 
    ENCLOSED BY '"' 
    LINES TERMINATED BY '\n'
    IGNORE 1 ROWS
    (StationId, Date, PM2_5, PM10, NO, NO2, NOx, NH3, CO, SO2, O3, Benzene, Toluene, Xylene, AQI, AQI_Bucket)
    SET PM2_5 = NULLIF(PM2_5, ''),
        PM10 = NULLIF(PM10, ''),
        NO = NULLIF(NO, ''),
        NO2 = NULLIF(NO2, ''),
        NOx = NULLIF(NOx, ''),
        NH3 = NULLIF(NH3, ''),
        CO = NULLIF(CO, ''),
        SO2 = NULLIF(SO2, ''),
        O3 = NULLIF(O3, ''),
        Benzene = NULLIF(Benzene, ''),
        Toluene = NULLIF(Toluene, ''),
        Xylene = NULLIF(Xylene, ''),
        AQI = NULLIF(AQI, ''),
        AQI_Bucket = NULLIF(AQI_Bucket, '');
""")

print("Data loaded successfully with NULL values!")

In [ ]:
# Execute SQL query to fetch data
cursor.execute("SELECT * FROM stations")
rows = cursor.fetchall()

columns = [desc[0] for desc in cursor.description]
station_df = pd.DataFrame(rows, columns=columns)

In [ ]:
# Execute SQL query to fetch data
cursor.execute("SELECT * FROM station_day")
rows = cursor.fetchall()

columns = [desc[0] for desc in cursor.description]
station_day_df = pd.DataFrame(rows, columns=columns)

conn.commit()
cursor.close()
conn.close()

In [ ]:
station_df

In [ ]:
# If can't connect to the database we can directly use these csv files

# station_df = pd.read_csv("stations.csv")
# station_day_df = pd.read_csv("station_day.csv")

In [ ]:
station_df.info()

In [ ]:
station_df.shape

In [ ]:
station_day_df

In [ ]:
station_day_df.info()

In [ ]:
station_day_df.shape


# ________________________________________________________________________________
# 3. making the station merged csv (main csv we will work with)

In [ ]:
# Joining the datasets as it makes analysing the data easier

# we dont need the name of the station and the satus if it is active or not so dropping these columns

station_df = station_df.drop(columns=["StationName","Status"])

station_df

In [ ]:
# Merging dataframes based on StationId column

station_merged_df = pd.merge(station_df, station_day_df, on="StationId")


# ________________________________________________________________________________
# 4. preliminary data analysis of station_merged_df  

In [ ]:
# 1. general shape
station_merged_df.shape

In [ ]:
# 2. some values
station_merged_df

In [ ]:
# 3. columns

station_merged_df.info()

In [ ]:
# 4 data distribution

station_merged_df.describe()

# AQL Category, Pollutants and Health Breakpoints

![aqi thresholds](https://user-images.githubusercontent.com/91218998/226107225-4e78b162-7844-43d9-88c4-17caf21f76da.png)

### From the above table we see that over the years from 2015 to 2020 the air has been moderately polluted 

In [ ]:
station_merged_df.isnull().sum()

* There are 21010 rows in the dataset where AQI, and AQI_Bucket are missing
* These rows are useless as they dont have our target value so we will remove them

In [ ]:
# we will only selecting values where isnull is not true (meaning the value is not null)

mask = station_merged_df["AQI"].isnull() == False
station_merged_df = station_merged_df[mask]

In [ ]:
station_merged_df.isnull().sum()

* The missing values now are numerical columns 
* The best way fill those values would be to replace these values with the mean of the column

In [ ]:
station_merged_df.shape

In [ ]:
station_merged_df.describe().loc["mean"]

In [ ]:
station_merged_df = station_merged_df.replace({

"PM2_5" : {np.nan:80.387649},
"PM10" :{np.nan:158.557614},
"NO": {np.nan:23.244401},
"NO2": {np.nan:35.118825},
"NOx": {np.nan:43.815139},
"NH3": {np.nan:28.654163},
"CO":  {np.nan:1.645650},
"SO2": {np.nan:12.212651},
"O3": {np.nan:38.320547},
"Benzene":  {np.nan:4.011256},
"Toluene": {np.nan:18.244392},
"Xylene":  {np.nan:3.368674}})
station_merged_df.isnull().sum()

### The lines below change date column into datetime pandas datatype and creates 3 columns (year, month, day) then drops the date column

In [ ]:
station_merged_df["Date"] = pd.to_datetime(station_merged_df["Date"])
station_merged_df["year"] = station_merged_df["Date"].dt.year
station_merged_df["month"] = station_merged_df["Date"].dt.month
station_merged_df["day"] = station_merged_df["Date"].dt.day
station_merged_df.drop('Date',axis=1,inplace=True)

<b> This is what our station_merged_df looks like after preliminary data analysis </b>

In [ ]:
station_merged_df.shape

In [ ]:
station_merged_df.info()

### in station merged df

| #  | Column     | Non-Null Count  | Dtype   |    meaning   |
|:-: | :------:     | :--------------:  | :-----:   | :---:  |
| 0  | StationId  | 108035 non-null | object  |    id of the station   |
| 1  | City       | 108035 non-null | object  |    city in which the station is located   |
| 2  | State      | 108035 non-null | object  |    state in which the station is located   |
| _  | Date       | 108035 non-null | object  |    date of the readings (now removed)   |
| 3  | PM2.5      | 86410 non-null  | float64 |    amount of said checmical in the air   |
| 4  | PM10       | 65329 non-null  | float64 |    amount of said checmical in the air   |
| 5  | NO         | 90929 non-null  | float64 |    amount of said checmical in the air   |
| 6  | NO2        | 91488 non-null  | float64 |    amount of said checmical in the air   |
| 7  | NOx        | 92535 non-null  | float64 |    amount of said checmical in the air   |
| 8  | NH3        | 59930 non-null  | float64 |    amount of said checmical in the air   |
| 9  | CO         | 95037 non-null  | float64 |    amount of said checmical in the air   |
| 10 | SO2        | 82831 non-null  | float64 |    amount of said checmical in the air   |
| 11 | O3         | 82467 non-null  | float64 |    amount of said checmical in the air   |
| 12 | Benzene    | 76580 non-null  | float64 |    amount of said checmical in the air   |
| 13 | Toluene    | 69333 non-null  | float64 |    amount of said checmical in the air   |
| 14 | Xylene     | 22898 non-null  | float64 |    amount of said checmical in the air   |
| 15 | AQI        | 87025 non-null  | float64 |    the air quality that day   |
| 16 | AQI_Bucket | 87025 non-null  | object  |    AQI catagory ranges from good to severe   |
| 17 | year       | 87025 non-null  | float64 |    year of the reading   |
| 18 | month      | 87025 non-null  | float64 |    month of the reading   |
| 18 | date       | 87025 non-null  | float64 |    day of the reading   |

In [ ]:
station_merged_df

In [ ]:
# writing the csv back
station_merged_df.to_csv("/Users/pulkitsoni/Air_Quality_Analysis_and_Prediction_for_Indian_Cities_and_States/station_merged.csv", index=False)

### things changed in station_merged_csv

* dropped all columns where aqi reading was none
* numerical missing values were replaced with means of the respective columns 
* dropped the date column and added day, month, year columns instead

### this is it for the preliminary analysis of station_merged_df now the fun part

# ________________________________________________________________________________

# 5. exploratory data analysis of station_merged_df (the main reason why i made this notebook)

### First lets see aqi trends from 2015-2020

In [ ]:
# -------------------------------------------------------------------------------------------------------------------
# 1. trend of aqi throughout the years 

plt.figure(figsize=(17,6), dpi=300)

font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 20}

mpl.rc('font', **font)

mpl.style.use("Solarize_Light2")
yearly_grouped = station_merged_df.groupby("year")["AQI"].mean().plot(kind="line",c="green", marker="o",title="trend of AQI troughout the years", ylabel="AQI Reading")


* the AQI has decreased steadily over the years esepecially from 2018
* the reason 2020 has such a low aqi is due to covid 19 lockdowns  
* you can read this news article https://blogs.worldbank.org/endpovertyinsouthasia/india-air-quality-has-been-improving-despite-covid-19-lockdown


In [ ]:
# -------------------------------------------------------------------------------------------------------------------
# 2. aqi trend of states in india

states_group = pd.DataFrame(station_merged_df.groupby("State")["AQI"].mean()).reset_index()
states_group = states_group.sort_values(by="AQI", ascending=False)
plt.figure(figsize=(20,7), dpi=300)
sns.set(font_scale=1.5)
sns.barplot(x='State', y='AQI', data=states_group).set(title ='AQI of sates')
plt.xticks(rotation=90)
plt.show()

* Gujrat has the most toxic air with aqi catagory of "Severe" even surpassing delhi
* Mizoram has the freshest air with an aqi of under 100

In [ ]:
# -------------------------------------------------------------------------------------------------------------------
# 3. aqi trend of cities in india

cities_group = pd.DataFrame(station_merged_df.groupby("City")["AQI"].mean()).reset_index()
cities_group = cities_group.sort_values(by="AQI", ascending=True)
mpl.style.use("Solarize_Light2")
plt.figure(figsize=(20,7), dpi=300)
sns.set(font_scale=2)
sns.barplot(x="City", y="AQI", data=cities_group).set(title="AQI of citites", xlabel="", ylabel="AQI")
plt.xticks(rotation=90)
plt.show()

* the aqi of Aizawl is the best (0-100) healthy
* the the other hand Ahemdabad is worst with the avrage aqi reaching (400+) severe

In [ ]:

# -------------------------------------------------------------------------------------------------------------------
# 4. Yearly/ monthly state_Aqi_from_2015_to_2020 function

def state_Aqi_from_2015_to_2020(required_state, year_or_month): # 1, 2

    '''
        This function shows the aqi trend of a state from 2016 to 2020 on a yearly or monthly basis

        steps:
        1. checks if year_or_month is 0 or 1 on these basis it shows the monthly aqi trend or yearly aqi trend
        2. gets the required state from the user
        3. makes mask and then makes required_state_df
        4. if 0 years_group is the mean() of all aqi readings from 2016 to 2020 else its all months from 2016 to 2020
        5. makes the bar/line plot
    '''

    if year_or_month == 0: # yearly aqi report

        required_state_mask = station_merged_df["State"] == required_state # 3
        required_state_df = station_merged_df[required_state_mask] # 3
        years_group = pd.DataFrame(required_state_df.groupby("year")["AQI"].mean()).reset_index() # 4
        mpl.style.use("Solarize_Light2")  # 5
        plt.figure(figsize=(20,5), dpi=300) # 5
        sns.set(font_scale=2) # 5
        sns.barplot(x="year", y="AQI", data=years_group).set(title=f"yearly AQI of {required_state}", xlabel="", ylabel="AQI") # 5

    elif year_or_month == 1: # monthly aqi report

        required_state_mask = station_merged_df["State"] == required_state # 3
        required_state_df = station_merged_df[required_state_mask] # 3
        months_group = pd.DataFrame(required_state_df.groupby("month")["AQI"].mean()).reset_index() # 4
        mpl.style.use("Solarize_Light2") # 5
        plt.figure(figsize=(20,5), dpi=300) # 5
        plt.xticks(rotation=45)
        plt.plot(["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEPT", "OCT","NOV", "DEC"], months_group["AQI"], marker="o", c="purple") # 5
        plt.title(f"aqi trend of {required_state} months from 2016-2020")
        plt.ylabel("AQI")

In [ ]:
state_list = station_merged_df.State.unique()
print("The number of states avaible are: ", len(state_list))
print()
print(state_list)

In [ ]:
required_state = "Andhra Pradesh"
year_or_month = 1
state_Aqi_from_2015_to_2020(required_state, year_or_month)

* the highest aqi of all months is in jan as BHOGI festival is celebrated in andra pradesh

In [ ]:
required_state = "Rajasthan"
year_or_month = 1
state_Aqi_from_2015_to_2020(required_state, year_or_month)

* the highest aqi of all months is in Nov as Diwali festival is celebrated in Rajasthan

In [ ]:
required_state = "Andhra Pradesh"
year_or_month = 0 # 0 for yeqrly report 1 for monthly
state_Aqi_from_2015_to_2020(required_state, year_or_month)

* highest aqi in 2017 with 2020 being the least toxic

In [ ]:
# -------------------------------------------------------------------------------------------------------------------
# 5. Yearly/ monthly city_Aqi_from_2015_to_2020 function

# Chennai

def city_Aqi_from_2015_to_2020(required_city, year_or_month): # 1, 2

    '''
        This function shows the aqi trend of a city from 2016 to 2020 on a yearly or monthly basis

        steps:
        1. checks if year_or_month is 0 or 1 on these basis it shows the monthly aqi trend or yearly aqi trend
        2. gets the required city from the user
        3. makes mask and then makes required_city_df
        4. if 0 years_group is the mean() of all aqi readings from 2016 to 2020 else its all months from 2016 to 2020
        5. makes the bar/line plot
    '''

    if year_or_month == 0: # yearly aqi report

        required_city_mask = station_merged_df["City"] == required_city # 3
        required_city_df = station_merged_df[required_city_mask] # 3
        years_group = pd.DataFrame(required_city_df.groupby("year")["AQI"].mean()).reset_index() # 4
        mpl.style.use("Solarize_Light2")  # 5
        plt.figure(figsize=(20,5), dpi=300) # 5
        sns.set(font_scale=2) # 5
        sns.barplot(x="year", y="AQI", hue="year", data=years_group, palette="coolwarm").set(title=f"yearly AQI of {required_city}", xlabel="", ylabel="AQI") # 5
        plt.legend().remove()  

    elif year_or_month == 1: # monthly report

        required_city_mask = station_merged_df["City"] == required_city # 3
        required_city_df = station_merged_df[required_city_mask] # 3
        months_group = pd.DataFrame(required_city_df.groupby("month")["AQI"].mean()).reset_index() # 4
        mpl.style.use("Solarize_Light2") # 5
        plt.figure(figsize=(20,5), dpi=300) # 5
        plt.xticks(rotation=45)
        plt.plot(["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEPT", "OCT","NOV", "DEC"], months_group["AQI"], marker="o", c="purple") # 5
        plt.title(f"aqi trend of {required_city} months from 2016-2020")
        plt.ylabel("AQI")

In [ ]:
city_list = station_merged_df.City.unique()
print("The number of cities avaible are: ", len(city_list))
print()
print(city_list)

In [ ]:
required_city = "Chennai"
year_or_month = 0 # 0 for yearly report 1 for monthly
city_Aqi_from_2015_to_2020(required_city, year_or_month)

* the aqi index of chennai has been steadily decreasing throughout the years

In [ ]:
required_city = "Chennai"
year_or_month = 1 # 0 for yeqrly report 1 for monthly
city_Aqi_from_2015_to_2020(required_city, year_or_month)

* jan seems to be the most toxic month for chennai as well

### correlation of toxic gasses with aqi

In [ ]:
correlation_table = station_day_df.select_dtypes(include=['number']).corr()
top_toxic_gases = correlation_table['AQI'].drop('AQI').sort_values(ascending=False)
top_toxic_gases

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 8), dpi=300)

sorted_correlation = correlation_table['AQI'].drop('AQI').sort_values(ascending=False)

# Convert to DataFrame
top_toxic_gases = sorted_correlation.reset_index()
top_toxic_gases.columns = ["Feature", "Correlation"]

# Create barplot with hue as the y variable
ax = sns.barplot(
    data=top_toxic_gases, 
    x="Correlation", 
    y="Feature", 
    hue="Feature",
    dodge=False,
    palette="coolwarm",
    legend=False
)

# Add annotations (text on bars)
for index, row in enumerate(top_toxic_gases.itertuples()):
    ax.text(row.Correlation, index, f"{row.Correlation:.2f}", 
            ha='left', va='center', fontsize=12, color='black', fontweight='bold')

# Labels & Title
plt.title("Correlation of Air Pollutants with AQI", fontsize=16)
plt.xlabel("Correlation with AQI", fontsize=14)
plt.ylabel("Toxic Gases", fontsize=14)
plt.grid(axis='x', linestyle='--', alpha=0.6)  # Optional: Add grid for readability

plt.show()

* so from the above barplot we see the best 3 gasses are as follows
* PM10 > PM2.5 > NOx | 0.89 > 0.81 > 0.51 respectively
* so i will be plotting their mean values throughout the months and years of avaible states and citites in india

In [ ]:
# -------------------------------------------------------------------------------------------------------------------
# 7. yearly/monthly state_toxic_gas_from_2015_2020 india function

def state_toxic_gas_from_2015_2020(required_state, year_or_month): # 1
    
    '''
        this function shows the trend of PM10, PM2.5, NOx in a state from 2015-2020

        steps:

        1. get the year_or_month and required_state from the user
        2. make a yearly trend report if 0 or a monthly trend report if 1
        3. extract the states rows from the df and remove all unwanted columns
        4. now from this df get the years/months for labels and extract mean values of all three pollutants from 2016-2020 in state
        5. make the multiple bar graph by setting up tick locations and putting values extracted above
    '''

    if year_or_month == 0: # 2

        required_state_mask = station_merged_df["State"] == required_state # 3 
        required_state_df = station_merged_df[required_state_mask] # 3 
        required_state_df = required_state_df[['PM2_5', 'PM10','NOx','AQI','year']] # 3

        year_group = required_state_df.groupby("year") # 4
        years = list(str(year) for year in year_group.groups.keys()) # 4
        state_PM10 = required_state_df.groupby("year")['PM10'].mean() # 4
        state_PM25 = required_state_df.groupby("year")['PM2_5'].mean() # 4
        state_NOx = required_state_df.groupby("year")['NOx'].mean() # 4

        mpl.style.use("Solarize_Light2")  # 5 
        plt.figure(figsize=(20,7), dpi=300) # 5 

        pm10_ticks = range(1, len(years)+1) # 5 
        pm25_ticks = [x+0.2 for x in pm10_ticks] # 5 
        NOx_ticks = [x-0.2 for x in pm10_ticks] # 5 

        plt.bar(pm10_ticks, state_PM10, width=0.2, label="PM10") # 5 
        plt.bar(pm25_ticks,state_PM25, width=0.2, label="PM2_5") # 5 
        plt.bar(NOx_ticks,state_NOx, width=0.2, label="NOx") # 5 

        plt.title(f'Pollution levels in {required_state}') # 5 
        plt.ylabel('Pollutant levels (in thier respective units)', fontsize=20) # 5 
        plt.xticks(pm10_ticks, years) # 5 
        plt.legend() # 5 
        plt.show() # 5 

    elif year_or_month == 1: # 2
        
        required_state_mask = station_merged_df["State"] == required_state # 3 
        required_state_df = station_merged_df[required_state_mask] # 3 
        required_state_df = required_state_df[['PM2_5', 'PM10','NOx','AQI','month']] # 3

        state_PM10 = list(required_state_df.groupby("month")['PM10'].mean()) # 4
        state_PM25 = list(required_state_df.groupby("month")['PM2_5'].mean()) # 4
        state_NOx = list(required_state_df.groupby("month")['NOx'].mean()) # 4

        mpl.style.use("Solarize_Light2")  # 5 
        plt.figure(figsize=(20,7)) # 5 

        pm10_ticks = range(1, len(state_PM10)+ 1) # 5 
        pm25_ticks = [x+0.2 for x in pm10_ticks] # 5 
        NOx_ticks = [x-0.2 for x in pm10_ticks] # 5 

        plt.bar(pm10_ticks, state_PM10, width=0.2, label="PM10") # 5 
        plt.bar(pm25_ticks,state_PM25, width=0.2, label="PM2.5") # 5 
        plt.bar(NOx_ticks,state_NOx, width=0.2, label="NOx") # 5 
        plt.title(f'Pollution levels in {required_state}') # 5 
        plt.ylabel('Pollutant levels (in thier respective units)', fontsize=20) # 5 

        if len(pm10_ticks) == 12:
            plt.xticks(pm10_ticks, ["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEPT", "OCT", "NOV", "DEC"]) # 5 
        elif len(pm10_ticks) != 12:
            plt.xticks(pm10_ticks, ["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "SEPT", "OCT", "NOV", "DEC"])
            
        plt.legend() # 5 
        plt.show() # 5


In [ ]:
state_list = station_merged_df.State.unique()
print("The number of states avaible are: ", len(state_list))
print()
print(state_list)

In [ ]:
required_state = 'Andhra Pradesh'
year_or_month = 0
state_toxic_gas_from_2015_2020(required_state, year_or_month)


* if you see the yearly aqi graph of andra pradesh 2017 had the highest aqi
* in our yearly toxic gass graph of andra pradesh 2017 has the highest values of these gasses
* this proves the high corellation of these 3 gasses with aqi 

In [ ]:
required_state = 'Andhra Pradesh'
year_or_month = 1
state_toxic_gas_from_2015_2020(required_state, year_or_month)

* the similar pattern can be seen in the monthly aqi graph of andrapradesh jan and dec peaked with the graph almost forming a u shape
* the monthly toxic gass graph of andra pradesh shows the same u shape with the aqi peaks in jan and dec 
* this proves the high correlation between these 3 gasses and aqi 

In [ ]:
# -------------------------------------------------------------------------------------------------------------------
# 7. yearly/monthly city_toxic_gas_from_2015_2020 india function

def city_toxic_gas_from_2015_2020(required_city, year_or_month): # 1
    
    '''
        this function shows the trend of PM10, PM2.5, NOx in a city from 2015-2020

        steps:

        1. get the year_or_month and required_state from the user
        2. make a yearly trend report if 0 or a monthly trend report if 1
        3. extract the cities rows from the df and remove all unwanted columns
        4. now from this df get the years/months for labels and extract mean values of all three pollutants from 2016-2020 in city
        5. make the multiple bar graph by setting up tick locations and putting values extracted above
    '''

    if year_or_month == 0: # 2

        required_city_mask = station_merged_df["City"] == required_city # 3 
        required_city_df = station_merged_df[required_city_mask] # 3 
        required_city_df = required_city_df[['PM2_5', 'PM10','NOx','AQI','year']] # 3

        year_group = required_city_df.groupby("year") # 4
        years = list(str(year) for year in year_group.groups.keys()) # 4
        city_PM10 = required_city_df.groupby("year")['PM10'].mean() # 4
        city_PM25 = required_city_df.groupby("year")['PM2_5'].mean() # 4
        city_NOx = required_city_df.groupby("year")['NOx'].mean() # 4

        mpl.style.use("Solarize_Light2")  # 5 
        plt.figure(figsize=(20,7), dpi=300) # 5 

        pm10_ticks = range(1, len(years)+1) # 5 
        pm25_ticks = [x+0.2 for x in pm10_ticks] # 5 
        NOx_ticks = [x-0.2 for x in pm10_ticks] # 5 

        plt.bar(pm10_ticks, city_PM10, width=0.2, label="PM10") # 5 
        plt.bar(pm25_ticks,city_PM25, width=0.2, label="PM2_5") # 5 
        plt.bar(NOx_ticks,city_NOx, width=0.2, label="NOx") # 5 

        plt.title(f'Pollution levels in {required_city}') # 5 
        plt.ylabel('Pollutant levels (in thier respective units)', fontsize=20) # 5 
        plt.xticks(pm10_ticks, years) # 5 
        plt.legend() # 5 
        plt.show() # 5 

    elif year_or_month == 1: # 2
        
        required_city_mask = station_merged_df["City"] == required_city # 3 
        required_city_df = station_merged_df[required_city_mask] # 3 
        required_city_df = required_city_df[['PM2_5', 'PM10','NOx','AQI','month']] # 3

        city_PM10 = list(required_city_df.groupby("month")['PM10'].mean()) # 4
        city_PM25 = list(required_city_df.groupby("month")['PM2_5'].mean()) # 4
        city_NOx = list(required_city_df.groupby("month")['NOx'].mean()) # 4

        mpl.style.use("Solarize_Light2")  # 5 
        plt.figure(figsize=(20,7)) # 5 

        pm10_ticks = range(1, len(city_PM10)+ 1) # 5 
        pm25_ticks = [x+0.2 for x in pm10_ticks] # 5 
        NOx_ticks = [x-0.2 for x in pm10_ticks] # 5 

        plt.bar(pm10_ticks, city_PM10, width=0.2, label="PM10") # 5 
        plt.bar(pm25_ticks,city_PM25, width=0.2, label="PM2_5") # 5 
        plt.bar(NOx_ticks,city_NOx, width=0.2, label="NOx") # 5 
        plt.title(f'Pollution levels in {required_city}') # 5 
        plt.ylabel('Pollutant levels (in thier respective units)', fontsize=20) # 5 

        if len(pm10_ticks) == 12:
            plt.xticks(pm10_ticks, ["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEPT", "OCT", "NOV", "DEC"]) # 5 
        elif len(pm10_ticks) != 12:
            plt.xticks(pm10_ticks, ["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "SEPT", "OCT", "NOV", "DEC"])
            
        plt.legend() # 5 
        plt.show() # 5

In [ ]:
city_list = station_merged_df.City.unique()
print("The number of cities avaible are: ", len(city_list))
print()
print(city_list)

In [ ]:
required_city = "Chennai"
year_or_month = 0
city_toxic_gas_from_2015_2020(required_city, year_or_month)

* if you check the aqi of chennai on the code above it shows a similar trend of it decreasing steadily over time
* this proves that the correlation of gasses with aqi is strong even on a city level
* lets check the monthly toxic gas trend in channai 

In [ ]:
required_city = "Chennai"
year_or_month = 1
city_toxic_gas_from_2015_2020(required_city, year_or_month)

* also a similar trend to monthly aqi of channai 

### by this analysis we can conclude that aqi of a city/ state highly depends on PM10, PM2.5, NOx levels in the air more insights are stated above

### finnaly lets conclude this notebook by showing the sates of these of states/cities on an indian map

In [ ]:

# -------------------------------------------------------------------------------------------------------------------
# 9. aqi of states on an india map

# states_geo_data =  gpd.read_file("india_state.geojson")
# sates_geo_data.head(2)

import json
import geopandas as gpd

# Load GeoJSON manually
with open("india_state.geojson", "r") as f:
    geojson_data = json.load(f)

# Convert to GeoDataFrame
states_geo_data = gpd.GeoDataFrame.from_features(geojson_data["features"])

# 🔹 Set CRS to WGS84 (EPSG:4326) required by Folium
states_geo_data.set_crs(epsg=4326, inplace=True)

# 🔹 Convert GeoDataFrame to JSON for Folium
states_geo_data = states_geo_data.to_json()

In [ ]:
india_state_map = folium.Map(location=[22.5937, 78.9629], zoom_start=5)

In [ ]:
states_geo_data

In [ ]:
india_state_map

In [ ]:
folium.Choropleth(
    geo_data=states_geo_data,
    data=station_merged_df,  
    columns=['State', 'AQI'],
    key_on='feature.properties.NAME_1',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.7,
    legend_name='AIR QUALITY INDEX OF STATES IN INDIA'
).add_to(india_state_map)

india_state_map

In [ ]:
folium.Choropleth(
    geo_data=states_geo_data,
    data=station_merged_df,  
    columns=['State', 'AQI'],
    key_on='feature.properties.NAME_1',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.7,
    legend_name='AIR QUALITY INDEX OF STATES IN INDIA'
).add_to(india_state_map)

india_state_map

* From the map above the norhtern india seems to be the most poulated 
* while jammu and kashmir and eastern southern and central india seem to be doing good

In [ ]:
# -------------------------------------------------------------------------------------------------------------------
# 9. aqi of cities on an india map

# Load GeoJSON manually
with open("/Users/pulkitsoni/Air_Quality_Analysis_and_Prediction_for_Indian_Cities_and_States/india_district.geojson", "r") as f:
    geojson_data = json.load(f)

# Convert to GeoDataFrame
cities_geo_data = gpd.GeoDataFrame.from_features(geojson_data["features"])

# 🔹 Set CRS to WGS84 (EPSG:4326), required for mapping
cities_geo_data.set_crs(epsg=4326, inplace=True)

# Check output (should be the same as `gpd.read_file()`)
print(cities_geo_data.head())


In [ ]:
india_city_map = folium.Map(location=[22.5937, 78.9629], zoom_start=5)

folium.Choropleth(
    geo_data=cities_geo_data,  # Ensure this file contains city boundaries
    data=station_merged_df,
    columns=['City', 'AQI'],
    key_on='feature.properties.NAME_1',  # Ensure city names match in both datasets
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.7,
    legend_name='AIR QUALITY INDEX OF CITIES IN INDIA'
).add_to(india_city_map)

india_city_map


* Online avaible json files only have data on chandighar and delhi which is why these are the only coloured ones
* Other cities have HIGH aqi but it wont show on this map as there data isnt avaible in the json file

### Take this map with a grain of salt the information above is misleading as some cities with higher aqi are lighter

# This is it for preliminary and exploratory analysis of station merged df
# ______________________________________________________________________________________________________________________

# PART 2 --> MODAL ANALYSIS 
<b> This is the second part of AQI_analysis_of_indian_states_and_cities PROJECT which proccess the data from station_merged csv and  trains the models. </b>

# objective 

<b> Use historical data of air quality readings from india to predict air quality index of a given region. </b>

# what does this notebook do?

<b> 

* https://www.diva-portal.org/smash/get/diva2:1681590/FULLTEXT02 in this reasearch paper the creators have only dumped the data into their machine learning models but did not explore the data itself. 

* The models they used achived (Ridge regression {mea: 27.907 rmse: 36.791, r2: 0.8089}) score so we will try to build a better model in this ipynb

</b>
<b> 

* The (xgb_model mea = 21.22, rmse = 37.011, r2 = 0.91) is made in this csv 
* This model will be used by PART 3 (3_main_AQI_predictor.pyw) to make predictions and show aqi 

# ________________________________________________________________________________

# 1. Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
import math
import joblib
%matplotlib inline

# ________________________________________________________________________________
# 2. reading the station_merged_csv

In [ ]:
station_merged_data = pd.read_csv("/Users/pulkitsoni/Air_Quality_Analysis_and_Prediction_for_Indian_Cities_and_States/station_merged.csv")

# ________________________________________________________________________________
# 3. preliminary and exploratory data analysis of station_merged_csv

checking if the station_merged_df 
0. things changed in the station_merged_csv
1. general shape 
2. some values
3. columns
4. data distribution


### 0. things changed in the station_merged_csv

* all of this stuff was changed in exploratory_analysis_of_aqi_in_india_1.ipynb

* dropped all columns where aqi reading was none
* numerical missing values were replaced with means of the respective columns 
* dropped the date column and added day, month, year columns instead

In [ ]:
# 1. general shape
station_merged_data.shape

In [ ]:
# 2. some values
station_merged_data


In [ ]:
# 3. columns
station_merged_data.info()

### in station merged df

| #  | Column     | Non-Null Count  | Dtype   |    meaning   |
|:-: | :------:     | :--------------:  | :-----:   | :---:  |
| 0  | StationId  | 108035 non-null | object  |    id of the station   |
| 1  | City       | 108035 non-null | object  |    city in which the station is located   |
| 2  | State      | 108035 non-null | object  |    state in which the station is located   |
| _  | Date       | 108035 non-null | object  |    date of the readings (now removed)   |
| 3  | PM2.5      | 86410 non-null  | float64 |    amount of said checmical in the air   |
| 4  | PM10       | 65329 non-null  | float64 |    amount of said checmical in the air   |
| 5  | NO         | 90929 non-null  | float64 |    amount of said checmical in the air   |
| 6  | NO2        | 91488 non-null  | float64 |    amount of said checmical in the air   |
| 7  | NOx        | 92535 non-null  | float64 |    amount of said checmical in the air   |
| 8  | NH3        | 59930 non-null  | float64 |    amount of said checmical in the air   |
| 9  | CO         | 95037 non-null  | float64 |    amount of said checmical in the air   |
| 10 | SO2        | 82831 non-null  | float64 |    amount of said checmical in the air   |
| 11 | O3         | 82467 non-null  | float64 |    amount of said checmical in the air   |
| 12 | Benzene    | 76580 non-null  | float64 |    amount of said checmical in the air   |
| 13 | Toluene    | 69333 non-null  | float64 |    amount of said checmical in the air   |
| 14 | Xylene     | 22898 non-null  | float64 |    amount of said checmical in the air   |
| 15 | AQI        | 87025 non-null  | float64 |    the air quality that day   |
| 16 | AQI_Bucket | 87025 non-null  | object  |    AQI catagory ranges from good to severe   |
| 17 | year       | 87025 non-null  | float64 |    year of the reading   |
| 18 | month      | 87025 non-null  | float64 |    month of the reading   |
| 18 | date       | 87025 non-null  | float64 |    day of the reading   |

In [ ]:
# 3. columns
station_merged_data.info()

In [ ]:
station_merged_data = station_merged_data.drop(columns=["StationId", "City", "State", "year", "month", "day"])

In [ ]:
station_merged_data

* we really dont need the StationId, City, State, year, month, date columns so we will remove them

In [ ]:
# 4. data distribution
station_merged_data.describe()

### most of the preliminary alanlysis has been done in the other ipynb this ipynb concentrates on the data distribution and model evaluation
# ___________________________________________________________________________________________________________

# Exploratory analysis:

### Heavy exploratory analysis of the dataset has already been done in the exploratory_analysis_of_aqi_in_india_1.ipynb i would highly prefer you read it first then come and check this csv out 

* lets check the distribution of values in each of the columns  

In [ ]:
# distribution of aqi from 2015-2020
plt.figure(figsize=(12, 6), dpi=300)  # High resolution with 300 DPI
sns.displot(station_merged_data, x="AQI", color="red")
plt.title("Distribution of AQI", fontsize=14)
plt.show()

* the avarage aqi across 87000 readings from station all over india has spike in the range of 100-200 and another spike in the range of 300-400

In [ ]:
# distribution of values of toxic gasses in the air
station_merged_data[[i for i in station_merged_data.columns if i not in ["AQI_Bucket","AQI", "year", "month", "day"]]].hist(bins=30, figsize=(20, 12), color="green", legend=True)
plt.show()

* benzene, xylene and etc are usually 0 so they are not a problem
* but PM10, PM2.5, NO2, NOx, NH3 have high levels of somewhere bettween 100-200 across 87000 readings from station all over india 

### lets check the coorelation of aqi with toxic gases using a heatmap

In [ ]:
correlation_table = station_merged_data.select_dtypes(include=['number']).corr()
top_toxic_gases = correlation_table['AQI'].drop('AQI').sort_values(ascending=False)
top_toxic_gases

In [ ]:
plt.figure(figsize=(15, 8), dpi=300)

# Take the top 3 toxic gases
top_toxic_gases = top_toxic_gases.reset_index()
top_toxic_gases.columns = ["Feature", "Correlation"]

# Create barplot
sns.barplot(data=top_toxic_gases, x="Correlation", y="Feature", hue="Feature", palette="coolwarm", dodge=False, legend=False)

# Labels & Title
plt.title("Correlation of Toxic Gases with AQI")
plt.xlabel("Correlation with AQI")
plt.ylabel("Toxic Gases")

plt.show()

* from the above table the top contenders for growth of aqi are PM2.5, PM19, NO, NO2, NOx, CO, SO2, NH3, TOULENE in this order

### this does it for exploratory analysis of aqi station readings across india now its time to test and train the models

# ________________________________________________________________________________________________________
# 4. data preproccessing and spiliting our dataset into target_y and features_x dfs

In [ ]:
# we wont need the aqi bucket column as we will use regression algorithms
station_merged_data = station_merged_data.drop(columns="AQI_Bucket")
target_y = station_merged_data["AQI"]
features_x = pd.DataFrame(station_merged_data.drop(columns="AQI"))

target_y

In [ ]:
features_x

In [ ]:
features_x.describe()

### why we are scaling our dataframe

* the data in features_x is on diffrent scale as in PM10 is in between 0.03 - 976.77 on the other hand CO is in range of 0.01 - 186.08
* if we feed this data to our models straight up it will be biased towards higher values in some models
* therefore we will standardize our value between -3 - 3 so our models generalize better to future values

In [ ]:
scaler = StandardScaler()

features_x = pd.DataFrame(scaler.fit_transform(features_x),columns=features_x.columns)
features_x.describe().round(2)

* notice how the mean and standard deviation of features_x has changed the mean is now 0 and standard deviation now 1

In [ ]:
features_x.hist(bins=30, figsize=(20,10), color="purple")

* notice how the x axis for most of the values has changed and everything has came down to a similar scale
* notice distribution of our data it is "right skewed" 
* the min and max values of our columns range from -ve to +ve so we will use yeo transform to normalise some of our columns

In [ ]:
# using yeo transform to transform features_x 

pt = PowerTransformer()

features_x_transformed = pt.fit_transform(features_x)
features_x_transformed = pd.DataFrame(features_x_transformed, columns=features_x.columns)
for col in features_x_transformed.columns:

    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,3))
    sns.histplot(features_x[col],kde=True,color="red", ax=ax1)
    ax1.set_title(str(col))
    sns.histplot(features_x_transformed[col],color="yellow",kde=True,ax=ax2)
    ax2.set_title(str(col+"_transformed"))
    plt.show()

In [ ]:
features_x = features_x_transformed

* the yeo transformation has done its wonders and most of the columns are now normalised making our data a good fit for XGBOOST reggressor multiple linear regression and svr

# _______________________________________________________________________________________________
# 5. using train test split and fitting our data into ml models of choice

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(features_x, target_y, random_state=0)

print(f"training data --> X_train: {X_train.shape}, Y_train: {Y_train.shape}")
print(f"testing data --> X_test: {X_test.shape}, Y_test: {Y_test.shape}")


In [ ]:
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score

In [ ]:
# fitting our data into a multiple linear regression model and storing the scores in results 

mlr_model = LinearRegression().fit(X_train, Y_train)
xgb_model = XGBRegressor().fit(X_train, Y_train)
svr_model = SVR(kernel='rbf', C=1.0, epsilon=0.1).fit(X_train, Y_train)
ridge_model = Ridge(alpha=1.0).fit(X_train, Y_train)
lasso_model = Lasso(alpha=1.0).fit(X_train, Y_train)
elastic_model = ElasticNet(alpha=1.0, l1_ratio=0.5).fit(X_train, Y_train)
dt_model = DecisionTreeRegressor().fit(X_train, Y_train)
rf_model = RandomForestRegressor(n_estimators=100).fit(X_train, Y_train)
gbm_model = GradientBoostingRegressor(n_estimators=100).fit(X_train, Y_train)
lgbm_model = LGBMRegressor().fit(X_train, Y_train)
catboost_model = CatBoostRegressor(verbose=0).fit(X_train, Y_train)
knn_model = KNeighborsRegressor(n_neighbors=5).fit(X_train, Y_train)
bayesian_model = BayesianRidge().fit(X_train, Y_train)

In [ ]:
mlr_model_prediction = mlr_model.predict(X_test)
xgb_model_prediction = xgb_model.predict(X_test)
svr_model_prediction = svr_model.predict(X_test)
ridge_model_prediction = ridge_model.predict(X_test)
lasso_model_prediction = lasso_model.predict(X_test)
elastic_model_prediction = elastic_model.predict(X_test)
dt_model_prediction = dt_model.predict(X_test)
rf_model_prediction = rf_model.predict(X_test)
gbm_model_prediction = gbm_model.predict(X_test)
lgbm_model_prediction = lgbm_model.predict(X_test)
catboost_model_prediction = catboost_model.predict(X_test)
knn_model_prediction = knn_model.predict(X_test)
bayesian_model_prediction = bayesian_model.predict(X_test)

# ________________________________________________________________________
# 6. evaluating our models

In [ ]:
mea_scores = []
cross_val_scores = []

# Training and Testing Scores
training_scores = [mlr_model.score(X_train, Y_train), xgb_model.score(X_train, Y_train), 
                   svr_model.score(X_train, Y_train), ridge_model.score(X_train, Y_train), 
                   lasso_model.score(X_train, Y_train), elastic_model.score(X_train, Y_train),
                   dt_model.score(X_train, Y_train), rf_model.score(X_train, Y_train), 
                   gbm_model.score(X_train, Y_train), lgbm_model.score(X_train, Y_train),
                   catboost_model.score(X_train, Y_train), knn_model.score(X_train, Y_train),
                   bayesian_model.score(X_train, Y_train)]

testing_scores = [metrics.r2_score(Y_test, mlr_model_prediction), metrics.r2_score(Y_test, xgb_model_prediction), 
                  metrics.r2_score(Y_test, svr_model_prediction), metrics.r2_score(Y_test, ridge_model_prediction),
                  metrics.r2_score(Y_test, lasso_model_prediction), metrics.r2_score(Y_test, elastic_model_prediction),
                  metrics.r2_score(Y_test, dt_model_prediction), metrics.r2_score(Y_test, rf_model_prediction),
                  metrics.r2_score(Y_test, gbm_model_prediction), metrics.r2_score(Y_test, lgbm_model_prediction),
                  metrics.r2_score(Y_test, catboost_model_prediction), metrics.r2_score(Y_test, knn_model_prediction),
                  metrics.r2_score(Y_test, bayesian_model_prediction)]

In [ ]:
cross_val_scores = [cross_val_score(mlr_model, features_x, target_y, cv=3, scoring='neg_mean_squared_error').mean(),
                    cross_val_score(xgb_model, features_x, target_y, cv=3, scoring='neg_mean_squared_error').mean(),
                    cross_val_score(svr_model, features_x, target_y, cv=3, scoring='neg_mean_squared_error').mean(),
                    cross_val_score(ridge_model, features_x, target_y, cv=3, scoring='neg_mean_squared_error').mean(),
                    cross_val_score(lasso_model, features_x, target_y, cv=3, scoring='neg_mean_squared_error').mean(),
                    cross_val_score(elastic_model, features_x, target_y, cv=3, scoring='neg_mean_squared_error').mean(),
                    cross_val_score(dt_model, features_x, target_y, cv=3, scoring='neg_mean_squared_error').mean(),
                    cross_val_score(rf_model, features_x, target_y, cv=3, scoring='neg_mean_squared_error').mean(),
                    cross_val_score(gbm_model, features_x, target_y, cv=3, scoring='neg_mean_squared_error').mean(),
                    cross_val_score(lgbm_model, features_x, target_y, cv=3, scoring='neg_mean_squared_error').mean(),
                    cross_val_score(catboost_model, features_x, target_y, cv=3, scoring='neg_mean_squared_error').mean(),
                    cross_val_score(knn_model, features_x, target_y, cv=3, scoring='neg_mean_squared_error').mean(),
                    cross_val_score(bayesian_model, features_x, target_y, cv=3, scoring='neg_mean_squared_error').mean()]

mea_scores = [mean_absolute_error(mlr_model_prediction, Y_test), mean_absolute_error(xgb_model_prediction, Y_test), 
              mean_absolute_error(svr_model_prediction, Y_test), mean_absolute_error(ridge_model_prediction, Y_test), 
              mean_absolute_error(lasso_model_prediction, Y_test), mean_absolute_error(elastic_model_prediction, Y_test), 
              mean_absolute_error(dt_model_prediction, Y_test), mean_absolute_error(rf_model_prediction, Y_test), 
              mean_absolute_error(gbm_model_prediction, Y_test), mean_absolute_error(lgbm_model_prediction, Y_test), 
              mean_absolute_error(catboost_model_prediction, Y_test), mean_absolute_error(knn_model_prediction, Y_test), 
              mean_absolute_error(bayesian_model_prediction, Y_test)]

print(training_scores)
print(testing_scores)
print(cross_val_scores)
print(mea_scores)

* in the code below cross_val_scores has the mean of all 3 cross_val_score of the model
* mea_scores is now at the correct place
* testing score are now at the correct place

In [ ]:
mlr_cv_scores = cross_val_score(mlr_model, X_train, Y_train, cv=5)
xgb_cv_scores = cross_val_score(xgb_model, X_train, Y_train, cv=5)
svr_cv_scores = cross_val_score(svr_model, X_train, Y_train, cv=5)
ridge_cv_scores = cross_val_score(ridge_model, X_train, Y_train, cv=5)
lasso_cv_scores = cross_val_score(lasso_model, X_train, Y_train, cv=5)
elastic_cv_scores = cross_val_score(elastic_model, X_train, Y_train, cv=5)
dt_cv_scores = cross_val_score(dt_model, X_train, Y_train, cv=5)
rf_cv_scores = cross_val_score(rf_model, X_train, Y_train, cv=5)
gbm_cv_scores = cross_val_score(gbm_model, X_train, Y_train, cv=5)
lgbm_cv_scores = cross_val_score(lgbm_model, X_train, Y_train, cv=5)
catboost_cv_scores = cross_val_score(catboost_model, X_train, Y_train, cv=5)
knn_cv_scores = cross_val_score(knn_model, X_train, Y_train, cv=5)
bayesian_cv_scores = cross_val_score(bayesian_model, X_train, Y_train, cv=5)

# Now calculate the average of each model's cross-validation scores
cross_val_scores = [sum(mlr_cv_scores)/len(mlr_cv_scores), 
                    sum(xgb_cv_scores)/len(xgb_cv_scores),
                    sum(svr_cv_scores)/len(svr_cv_scores),
                    sum(ridge_cv_scores)/len(ridge_cv_scores),
                    sum(lasso_cv_scores)/len(lasso_cv_scores),
                    sum(elastic_cv_scores)/len(elastic_cv_scores),
                    sum(dt_cv_scores)/len(dt_cv_scores),
                    sum(rf_cv_scores)/len(rf_cv_scores),
                    sum(gbm_cv_scores)/len(gbm_cv_scores),
                    sum(lgbm_cv_scores)/len(lgbm_cv_scores),
                    sum(catboost_cv_scores)/len(catboost_cv_scores),
                    sum(knn_cv_scores)/len(knn_cv_scores),
                    sum(bayesian_cv_scores)/len(bayesian_cv_scores)]

In [ ]:
cross_val_scores

In [ ]:
model_names = ["mlr_model", "xgb_model","svr_model", "ridge_model", "lasso_model", "elastic_model", "dt_model", "rf_model", "gbm_model", "lgbm_model", "catboost_model", "knn_model", "bayesian_model"]
final_scores_df = pd.DataFrame({"model_names":model_names ,"training_scores": training_scores,"testing_scores": testing_scores, "mea_scores": mea_scores, "cross_val_scores": cross_val_scores}).round(2)
final_scores_df = final_scores_df.sort_values(by="cross_val_scores")
final_scores_df

* ✅ 1. Random Forest (rf_model)
* Training Score: 0.99
* Testing Score: 0.92
* MAE: 20.82 (Lowest)
* Cross-Val Score: 0.92

* ✅ 2. LightGBM (lgbm_model)
* Training Score: 0.94
* Testing Score: 0.92
* MAE: 21.45
* Cross-Val Score: 0.92

* ✅ 3. CatBoost (catboost_model)
* Training Score: 0.95
* Testing Score: 0.92
* MAE: 20.91
* Cross-Val Score: 0.92


* Random Forest is the best model overall (lowest MAE and high scores).
* LightGBM and CatBoost are very close competitors.
* Decision Tree (dt_model) is overfitting (Training Score: 1.00, but Testing Score: 0.84).
* Elastic Net, MLR, Ridge, and Bayesian models perform the worst across all metrics.

In [ ]:
mpl.style.use("Solarize_Light2")
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2,figsize=(20, 8))

ax1.plot(model_names,final_scores_df["training_scores"],color="orange", marker="o")
ax1.set_title("training_scores")

ax2.plot(model_names,final_scores_df["testing_scores"],color="green", marker="o")
ax2.set_title("testing_scores")

ax3.plot(model_names,final_scores_df["mea_scores"],color="Blue", marker="o")
ax3.set_title("mea_scores")

ax4.plot(model_names,final_scores_df["cross_val_scores"],color="purple", marker="o")
ax4.set_title("cross_val_scores")

plt.show()

In [ ]:
rf_model_mse = metrics.mean_squared_error(rf_model_prediction, Y_test)
rf_model_mea = metrics.mean_absolute_error(rf_model_prediction, Y_test)
rf_model_rmse = math.sqrt(rf_model_mse)

print("The mean absolute error of the model is : ",rf_model_mea.round(2))
print("The mean squared error of the model is : ",rf_model_mse.round(2))
print("The mean root mean squared error of the model is : ",rf_model_rmse)

### the xgbRegressor model dosent need the data to be scaled and yeo transformed so lets try find out how good that model is 

In [ ]:
Y = station_merged_data["AQI"]
X = station_merged_data.drop(columns=["AQI"])

# these are spllited versions of nomral data which is not scaled not yeo transformed

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0)

normal_rf_model = RandomForestRegressor().fit(X_train, Y_train)
normal_rf_model_prediction = normal_rf_model.predict(X_test)

print(" --- stats of normal_rf_model --- ")
print("Training score: ", round(normal_rf_model.score(X_train, Y_train), 2))
print("Testing score: ", round(metrics.r2_score(normal_rf_model_prediction, Y_test), 2))
print("Mean absolute error: ", round(metrics.mean_absolute_error(normal_rf_model_prediction, Y_test), 2))
print("Mean squared error: ", round(metrics.mean_squared_error(normal_rf_model_prediction, Y_test), 2))
print("Root mean squared error: ", round(math.sqrt(metrics.mean_squared_error(normal_rf_model_prediction, Y_test)), 2))


# ______________________________________________________________________________________________
# 7. conclusion

### from the above tables and plots we can conclude these things
* RandomForestRegressor is the fastest and the best model with a cross_val_score of 0.92 and testing score of 0.92
* the best mea_score i could get was 21.45(LGBM) which is not bad as aqi ranges from 0 - 400 but it can still be worked on
* Elastic Net, MLR, Ridge, and Bayesian models perform the worst across all metrics.

# 8. has the main goal of the notebook been achived?

* The best model in the research paper was (Ridge regression mea = 27.907, rmse = 36.791, r2 = 0.8089) on city_data csv
* my best model in this notebook was (Randon_Forest mea = 20.73, rmse = 36.97, r2 = 0.91) on station_data csv

* the city_data csv has only 30000 rows while station_data csv has 100000 which is way more data 
* my model has achived better mea and r2 scores on a bigger scale of data making it better than the ridge reggression model made in the research paper

* but my is not the best and can still be upgraded 

# _________________________________________________________________________________________________________
# 9. saving the model into a binary file so we can use it in the GUI program

### since the xgbr model dosent need the data to be yeo transfoemed and scaled we will fit the normal data into it

In [ ]:
rf_model_file_path = "/Users/pulkitsoni/Air_Quality_Analysis_and_Prediction_for_Indian_Cities_and_States/rf_model"
joblib.dump(normal_rf_model, rf_model_file_path)


# Feature Importance

In [ ]:
# Get the feature importances
importances = normal_rf_model.feature_importances_

# Get the feature names
features = X.columns

# Sort the features by importance
indices = np.argsort(importances)[::-1]

# Plot the feature importances
plt.figure(figsize=(10, 6))
plt.bar(range(len(importances)), importances[indices], align="center")
plt.xticks(range(len(importances)), features[indices], rotation=90)
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.title('Feature Importance (RandomForestRegressor)')
plt.show()